In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import tqdm
import matplotlib.pyplot as plt
from scipy.ndimage import grey_erosion, grey_dilation, grey_opening, grey_closing
from skimage.feature import canny
from PIL import Image
%config Completer.use_jedi=False

In [2]:
annot = pd.read_csv('annotations.csv', index_col=False)
annot

,filepath,type,energy
0,train\ER\-0.003018522635102272__CYGNO_60_40_ER...,ER,30
1,train\ER\-0.0031084376387298107__CYGNO_60_40_E...,ER,3
2,train\ER\-0.003241758095100522__CYGNO_60_40_ER...,ER,3
3,train\ER\-0.0032767392694950104__CYGNO_60_40_E...,ER,10
4,train\ER\-0.004240238573402166__CYGNO_60_40_ER...,ER,3
...,...,...,...
13399,train\NR\3.1387999057769775__CYGNO_60_40_He_NR...,He,1
13400,train\NR\3.140399932861328__CYGNO_60_40_He_NR_...,He,20
13401,train\NR\3.1407999992370605__CYGNO_60_40_He_NR...,He,20
13402,train\NR\3.1412999629974365__CYGNO_60_40_He_NR...,He,6


In [3]:
def preprocess_img(path):
    orig = cv2.imread(path, 0)[188:388, 188: 388]
    img = orig = orig * (orig > np.quantile(orig, 0.8))
    img = grey_opening(img, size=(3,3))
    img = grey_closing(img, size=(3,3))
    img = orig * (img > 0)
    if np.count_nonzero(img) == 0:
        img = orig * (orig >= orig.max())
        img = grey_dilation(img, size=(3,3)).astype(bool) * orig
    else:
        img = grey_dilation(img, size=(17,17)).astype(bool) * orig
    return img

10207

In [5]:
suffix = 'mod2'
train_out = f'train_{suffix}'
test_out = f'test_{suffix}'

In [45]:
if not os.path.exists(train_out): os.mkdir(train_out)
for i in tqdm.trange(len(annot)):
    fp = annot['filepath'].iloc[i]
    cv2.imwrite(fp.replace('train\\ER', train_out).replace('train\\NR', train_out), preprocess_img(fp))

100%|███████████████████████████████████████████████████████████████████████████| 13404/13404 [01:26<00:00, 154.18it/s]


In [7]:
if not os.path.exists(test_out): os.mkdir(test_out)
for folder in ('public_test', 'private_test_final'):
    for fn in tqdm.tqdm(os.listdir(folder)):
        cv2.imwrite(f'{test_out}\\{fn}', preprocess_img(f'{folder}\\{fn}'))

100%|████████████████████████████████████████████████████████████████████████████| 15058/15058 [02:36<00:00, 96.39it/s]
